## Load Libraries

In [63]:
import os
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

### Intitiate Spark Session

In [64]:
spark = SparkSession.builder.appName("June 2021").getOrCreate()

spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set("spark.sql.session.timeZone", "IST")
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

key,value
spark.sql.legacy....,LEGACY


## Define Directories

In [65]:
PROJECT_DIR = "/media/bharathkancharla/techspace/Data Science World/Challenges/Analytical Vidhya/JOB-A-THON - June 2021"
DATA_DIR = os.path.join(PROJECT_DIR, "data")

In [66]:
ref_date = "28-May-2018"

In [67]:
from datetime import timedelta
def date_shift(ref, noofdayshift, as_string=False):
    shift_date = pd.to_datetime(ref) + timedelta(days=noofdayshift)
    if as_string:
        return str(shift_date)
    else:
        return shift_date

## Load the data

In [68]:
visitor_logs = spark.read.csv(os.path.join(DATA_DIR, "VisitorLogsData.csv"), header=True, inferSchema=True)
users = spark.read.csv(os.path.join(DATA_DIR, "userTable.csv"), header=True, inferSchema=True)

visitor_logs

webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
WI10000050298,2018-05-07 04:28:...,pr100631,null,null,Chrome Mobile,Android,Chennai,India
WI10000025922,2018-05-13 07:26:...,pr100707,null,null,Chrome,Windows,null,Taiwan
WI100000204522,2018-05-11 11:43:...,pr100030,null,click,Chrome,windows,Gurgaon,India
WI10000011974,2018-05-13 15:20:...,Pr100192,null,CLICK,Chrome,Windows,,
WI100000441953,2018-05-08 20:44:...,Pr100762,null,click,Chrome,mac os x,Iselin,United States
WI10000042224,2018-05-24 13:10:...,pr100109,null,click,Chrome,Mac OS X,null,Taiwan
WI100000532584,2018-05-23 14:52:...,pr100020,null,click,Opera,Windows,Baku,Azerbaijan
WI100000377250,2018-05-13 03:13:...,Pr101495,null,CLICK,Chrome,Windows,Bayan Lepas,Malaysia
WI10000031378,2018-05-10 02:04:...,Pr100013,null,click,Chrome,Windows,Chakwal,Pakistan
WI100000278874,2018-05-25 13:40:...,Pr101701,null,PAGELOAD,Chrome,Windows,,


In [69]:
users

UserID,Signup Date,User Segment
U133159,2018-04-14 07:01:...,C
U129368,2017-12-02 09:38:...,B
U109654,2013-03-19 11:38:...,B
U108998,2018-01-18 08:29:...,C
U131393,2018-03-27 08:05:...,B
U101889,2018-01-24 12:50:...,B
U113233,2017-03-18 09:27:...,B
U115728,2017-08-18 11:40:...,B
U132899,2018-03-14 08:32:...,B
U107492,2018-04-10 20:42:...,B


## EDA

- Filter out the visitor log data which doesn't contain `UserID`

In [70]:
visitor_logs = visitor_logs.filter(col("UserID").isNotNull())

- replace empty string to null for consistency across the data set

In [71]:
visitor_logs = visitor_logs.select([when(trim(col(c))=="",None).otherwise(col(c)).alias(c) for c in visitor_logs.columns])
users = users.select([when(trim(col(c))=="",None).otherwise(col(c)).alias(c) for c in users.columns])
visitor_logs.orderBy("UserID")

webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
WI10000020634,2018-05-18 10:39:...,Pr101097,U100002,null,Chrome Mobile,Android,Salvador,Brazil
WI10000020634,2018-05-18 10:39:...,pr100258,U100002,null,Chrome Mobile,Android,Salvador,Brazil
WI10000020634,2018-05-18 10:35:...,Pr100258,U100002,pageload,Chrome Mobile,Android,Salvador,Brazil Salvador
WI10000020634,2018-05-18 10:43:...,Pr100258,U100002,pageload,Chrome Mobile,Android,Salvador,Brazil
WI10000020634,1526639986717000000,Pr100258,U100002,click,Chrome Mobile,Android,Salvador,Brazil
WI100000392911,2018-05-25 02:16:...,pr100079,U100003,click,Chrome,Windows,Kolkata,India
WI100000392911,2018-05-25 02:17:...,null,U100003,click,Chrome,Windows,Kolkata,India Kolkata
WI100000392911,2018-05-25 02:15:...,Pr100079,U100003,pageload,Chrome,Windows,Kolkata,India
WI100000979909,2018-05-13 14:14:...,Pr100178,U100003,null,Chrome,Windows,Kolkata,India
WI10000097797,2018-05-16 17:55:...,pr100355,U100004,click,Chrome,windows,Urbana,United States


In [72]:
from datetime import datetime
datetime.now()

datetime.datetime(2021, 7, 4, 17, 30, 54, 345726)

In [73]:
visitor_logs = visitor_logs.filter(col("VisitDateTime").isNotNull())

In [74]:
visitor_logs = visitor_logs.withColumn("VisitDateTime", when(col("VisitDateTime").contains("-"), to_date(col("VisitDateTime"))).\
    otherwise(to_timestamp(col('VisitDateTime')/1000000000).cast('date')))

visitor_logs

webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
WI100000112772,2018-05-15,Pr100017,U106593,click,Chrome Mobile,Android,null,null
WI1000009977,2018-05-23,Pr101008,U108297,null,Chrome Mobile,Android,Delhi,India
WI100000223826,2018-05-10,Pr100241,U132443,null,Firefox,Windows,null,India
WI10000021998,2018-05-08,pr100495,U134616,click,Chrome,Windows,Cottage Grove,United States
WI10000036281,2018-05-11,Pr100363,U130784,click,Chrome,Chrome OS,New Delhi,India
WI100000110130,2018-05-19,pr100340,U120983,click,Chrome,Windows,Mumbai,India
WI10000014616,2018-05-19,Pr100166,U120287,click,Chrome,Windows,null,null
WI100000102012,2018-05-07,pr101042,U124307,CLICK,Chrome,Mac OS X,null,India
WI10000018039,2018-05-23,Pr101042,U113937,CLICK,Safari,mac os x,Dailekh,Nepal
WI1000001200,2018-05-13,Pr101042,U115735,click,Chrome,Windows,Khammam,India


In [75]:
visitor_logs.printSchema()

root
 |-- webClientID: string (nullable = true)
 |-- VisitDateTime: date (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- UserID: string (nullable = true)
 |-- Activity: string (nullable = true)
 |-- Browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)



In [76]:
spark.conf.get('spark.sql.session.timeZone')

'IST'

## Filter Visitor data for 21 days

In [77]:
visitor_logs = visitor_logs.withColumn("end_date", date_add(to_date(lit(ref_date), format="dd-MMM-yyyy"), -1))
visitor_logs = visitor_logs.withColumn("start_date", date_add(to_date(lit(ref_date), format="dd-MMM-yyyy"), -21))

visitor_logs

webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country,end_date,start_date
WI100000112772,2018-05-15,Pr100017,U106593,click,Chrome Mobile,Android,null,null,2018-05-27,2018-05-07
WI1000009977,2018-05-23,Pr101008,U108297,null,Chrome Mobile,Android,Delhi,India,2018-05-27,2018-05-07
WI100000223826,2018-05-10,Pr100241,U132443,null,Firefox,Windows,null,India,2018-05-27,2018-05-07
WI10000021998,2018-05-08,pr100495,U134616,click,Chrome,Windows,Cottage Grove,United States,2018-05-27,2018-05-07
WI10000036281,2018-05-11,Pr100363,U130784,click,Chrome,Chrome OS,New Delhi,India,2018-05-27,2018-05-07
WI100000110130,2018-05-19,pr100340,U120983,click,Chrome,Windows,Mumbai,India,2018-05-27,2018-05-07
WI10000014616,2018-05-19,Pr100166,U120287,click,Chrome,Windows,null,null,2018-05-27,2018-05-07
WI100000102012,2018-05-07,pr101042,U124307,CLICK,Chrome,Mac OS X,null,India,2018-05-27,2018-05-07
WI10000018039,2018-05-23,Pr101042,U113937,CLICK,Safari,mac os x,Dailekh,Nepal,2018-05-27,2018-05-07
WI1000001200,2018-05-13,Pr101042,U115735,click,Chrome,Windows,Khammam,India,2018-05-27,2018-05-07


In [78]:
visitor_logs = visitor_logs.filter((col("VisitDateTime") >=col("start_date")) & (col("VisitDateTime") <= col("end_date")))
visitor_logs = visitor_logs.drop("end_date", "start_date")

In [79]:
cols_to_lower = ["ProductID","Activity", "Browser", "OS", "City", "Country"]

for c in cols_to_lower:
    visitor_logs = visitor_logs.withColumn(c, lower(col(c)))

In [80]:
for c in ["Activity", "Browser", "OS", "City", "Country"]:
    print(f"Unique of the column: {c}")
    visitor_logs.select(c).distinct().show()

Unique of the column: Activity
+--------+
|Activity|
+--------+
|    null|
|   click|
|pageload|
+--------+

Unique of the column: Browser
+--------------------+
|             Browser|
+--------------------+
|            chromium|
|       mobile safari|
|             firefox|
|      firefox mobile|
|       chrome mobile|
|          apple mail|
|              chrome|
|          qq browser|
|      yandex browser|
|                edge|
|          uc browser|
|             maxthon|
|        opera mobile|
|         amazon silk|
|mobile safari ui/...|
|             coc coc|
|    samsung internet|
|   chrome mobile ios|
|         firefox ios|
|chrome mobile web...|
+--------------------+
only showing top 20 rows

Unique of the column: OS
+---------+
|       OS|
+---------+
|  android|
|   ubuntu|
|   fedora|
|    linux|
|chrome os|
|      ios|
|    tizen|
| mac os x|
|  windows|
+---------+

Unique of the column: City
+-------------+
|         City|
+-------------+
| nieuw-vennep|
|        s

In [81]:
timeline = pd.DataFrame(pd.date_range(start=date_shift(ref_date, -21),end=date_shift(ref_date, -1)))
users_df = users.select("UserID").distinct().toPandas()


timeline['key'] = 0
users_df['key'] = 0

userTimeline = users_df.merge(timeline, on='key', how='outer')
userTimeline = userTimeline.rename(columns={0: "VisitDateTime"})
userTimeline = userTimeline.drop("key", axis=1)

In [82]:
spark.conf.set("spark.sql.session.timeZone", "UTC")
userTimeline = spark.createDataFrame(userTimeline)

In [83]:
spark.conf.set("spark.sql.session.timeZone", "IST")
userTimeline = userTimeline.withColumn("VisitDateTime", to_date(col("VisitDateTime").cast("date")))

In [84]:
merged_data = userTimeline.join(visitor_logs, on= ["UserID", "VisitDateTime"], how="left")

In [85]:
final_merged_data = merged_data.join(users, on= "UserID", how="left").orderBy("UserID", desc("VisitDateTime"))
final_merged_data = final_merged_data.withColumn("last_7_days", date_add(to_date(lit(ref_date), format="dd-MMM-yyyy"), -7))
final_merged_data = final_merged_data.withColumn("last_15_days", date_add(to_date(lit(ref_date), format="dd-MMM-yyyy"), -15))
final_merged_data = final_merged_data.withColumn("Is_Active", when(col("webClientID").isNotNull(), 1).otherwise(0))
final_merged_data

UserID,VisitDateTime,webClientID,ProductID,Activity,Browser,OS,City,Country,Signup Date,User Segment,last_7_days,last_15_days
U100002,2018-05-27,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-26,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-25,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-24,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-23,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-22,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-21,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-20,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-19,null,null,null,null,null,null,null,2018-04-05 21:11:...,B,2018-05-21,2018-05-13
U100002,2018-05-18,WI10000020634,pr100258,null,chrome mobile,android,salvador,brazil,2018-04-05 21:11:...,B,2018-05-21,2018-05-13


In [86]:
final_merged_data.select("UserID").distinct().count()

34050

In [119]:
import sys
windowval = (Window.partitionBy('UserID').orderBy(desc('VisitDateTime'))
             .rangeBetween(Window.unboundedPreceding, 0))
final_merged_data = final_merged_data.withColumn('cum_sum', sum('Is_Active').over(windowval))

final_merged_data

UserID,VisitDateTime,webClientID,ProductID,Activity,Browser,OS,City,Country,Signup Date,User Segment,last_7_days,last_15_days,Is_Active,cum_sum,cumsum
U100364,2018-05-27,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-26,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-25,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-24,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-23,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-22,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-21,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-20,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-19,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-18,null,null,null,null,null,null,null,2017-07-23 16:32:...,B,2018-05-21,2018-05-13,0,0,0


In [134]:
final_merged_data.withColumn("Signup Date", to_date("Signup Date",format="dd-MMM-yyyy"))

UserID,VisitDateTime,webClientID,ProductID,Activity,Browser,OS,City,Country,Signup Date,User Segment,last_7_days,last_15_days,Is_Active,cum_sum,cumsum
U100364,2018-05-27,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-26,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-25,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-24,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-23,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-22,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-21,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-20,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-19,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0
U100364,2018-05-18,null,null,null,null,null,null,null,null,B,2018-05-21,2018-05-13,0,0,0


In [137]:
# User_Vintage

final_merged_data\
    .drop_duplicates(["UserID"])\
    .select("UserID", "Signup Date")\
    .withColumn("User_Vintage", datediff(to_date(lit(ref_date),format="dd-MMM-yyyy"),
    to_date("Signup Date")))

UserID,Signup Date,User_Vintage
U100364,2017-07-23 16:32:...,309
U100499,2017-02-08 10:27:...,474
U100608,2017-03-01 08:13:...,453
U100611,2016-06-30 10:22:...,697
U100754,2016-08-24 19:18:...,642
U100999,2013-04-03 16:32:...,1881
U102315,2018-03-03 15:38:...,86
U103221,2018-04-03 09:28:...,55
U103279,2018-01-27 11:23:...,121
U103647,2018-02-17 19:03:...,100


In [112]:
# Most_Active_OS
counts = final_merged_data.groupBy(["UserID", "OS"]).count()
counts = counts.filter(col("OS").isNotNull()).orderBy("UserID", desc("count"))
win = Window().partitionBy('UserID').orderBy(col('count').desc())
result = (counts
          .withColumn('row_num', row_number().over(win))
          .where(col('row_num') == 1)
          .select('UserID', 'OS')
         )
result.orderBy("UserID").withColumnRenamed("OS", "Most_Active_OS")

UserID,Most_Active_OS
U100002,android
U100003,windows
U100004,windows
U100005,android
U100006,android
U100007,windows
U100008,android
U100009,android
U100012,mac os x
U100013,mac os x


In [115]:
# Recently_Viewed_Product
final_merged_data.filter(col("Activity") == "pageload")\
    .filter(col("ProductID").isNotNull())\
    .orderBy("UserID", desc("VisitDateTime"))\
    .drop_duplicates(["UserID"])\
    .select("UserID", "ProductID")

UserID,ProductID
U100499,pr100051
U100608,pr100341
U100611,pr100138
U100754,pr100166
U100999,pr100017
U102315,pr100027
U103221,pr100483
U103647,pr100166
U103859,pr100321
U103874,pr101042


In [90]:
temp1 = final_merged_data.filter((col("VisitDateTime") >= col("last_7_days")) & (col("webClientID").isNotNull()))\
    .groupBy("UserID").agg(countDistinct("VisitDateTime").alias("No_of_days_Visited_7_Days"))\
        .orderBy("UserID")
temp1

UserID,No_of_days_Visited_7_Days
U100003,1
U100004,1
U100005,1
U100006,1
U100008,6
U100009,4
U100012,2
U100013,3
U100014,2
U100015,1


In [92]:
final_merged_data.filter((col("VisitDateTime") >= col("last_7_days")) & (col("webClientID").isNotNull()))\
    .groupBy(["UserID","Activity"]).count()

UserID,Activity,count
U100031,pageload,8
U105360,pageload,4
U117118,null,1
U104675,pageload,2
U109048,pageload,1
U104238,null,27
U111729,click,2
U104888,pageload,2
U113579,pageload,1
U110149,null,4


In [131]:
# Pageloads_last_7_days & Clicks_last_7_days
test = final_merged_data.filter((col("VisitDateTime") >= col("last_7_days")) & (col("webClientID").isNotNull()))\
    .groupBy("UserID").pivot("Activity").count().drop("null").orderBy("UserID")

test.filter((col("click")-col("pageload")>0))

UserID,click,pageload
U100003,2,1
U100008,23,21
U100012,14,5
U100014,2,1
U100015,8,3
U100018,21,7
U100019,15,5
U100026,2,1
U100027,36,15
U100030,4,2


In [127]:
# Most_Viewed_product_15_Days
last_15_days = final_merged_data.filter((col("VisitDateTime") >= col("last_15_days")) & (col("ProductID").isNotNull()))\
    .orderBy("UserID", "VisitDateTime")

counts = last_15_days.groupBy(["UserID", "ProductID"]).count()
counts = counts.filter(col("ProductID").isNotNull()).orderBy("UserID", desc("count"))
win = Window().partitionBy('UserID').orderBy(col('count').desc())
most_viewed_product = (counts
          .withColumn('row_num', row_number().over(win))
          .where(col('row_num') == 1)
          .drop_duplicates(["UserID"])
          .select('UserID', 'ProductID')
         )
most_viewed_product = most_viewed_product.withColumnRenamed("ProductID", "Most_Viewed_product_15_Days")
most_viewed_product

UserID,Most_Viewed_product_15_Days
U100364,pr100152
U100499,pr101042
U100608,pr100283
U100611,pr100138
U100754,pr101062
U100999,pr100017
U102315,pr100027
U103221,pr100483
U103647,pr100390
U103859,pr100252


In [129]:
no_products_viewed = last_15_days.groupBy("UserID").agg(expr('count(distinct ProductID)').alias('No_Of_Products_Viewed_15_Days'))
no_products_viewed

UserID,No_Of_Products_Viewed_15_Days
U121194,3
U117953,4
U100499,50
U104625,6
U109554,4
U128906,1
U100754,8
U112607,2
U105501,5
U127748,2
